In [1]:
import sys
sys.path.append('/home/users/piyushb/projects/correlation-GAN/')

In [2]:
import warnings
warnings.simplefilter('ignore')

import sys
import pdb
import numpy as np
import pandas as pd
from os.path import join
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm

from utils.constants import DATA_DIR
from utils.data import load_data_config, load_dataset, sample_subset_of_dataset

In [3]:
def seperate_real_and_categorical_columns(data):
    
    cols = data.columns
    num_cols = list(data._get_numeric_data().columns)
    non_num_cols = list(set(cols) - set(num_cols))

    num_data = data[num_cols]
    non_num_data = data[non_num_cols]
    data = pd.concat([num_data, non_num_data], axis=1)
    
    return data


def convert_target_column_to_categorical(target_data, basis='median'):
    
    if basis == 'median':
        mid_value = target_data.median()
    
    target_data_copy = target_data.copy()
    target_data.loc[target_data_copy <= mid_value]  = 'low'
    target_data.loc[target_data_copy > mid_value]  = 'high'
    
    return target_data
    

def seperate_out_target_variable(data_config, data, target_column):
    
    cols = list(data.columns)
    cols.pop(cols.index(target_column))
    non_target_data = data[cols]
    target_data = data[target_column]
    if data_config['target_variable_type'] == 'real':
        target_data = convert_target_column_to_categorical(target_data)
    new_data = pd.concat([non_target_data, target_data], axis=1)
    
    return new_data


def replace_nan_by_mean(dataset, num_real):
    # For numeric columns
    for i in range(num_real):
        col = dataset.iloc[:, i]
        col = col.fillna(col.mean())
        dataset.iloc[:, i] = col
    
    # For non-numeric columns
    for i in range(num_real, dataset.shape[1]):
        col = dataset.iloc[:, i]
        temp = (col.values).astype('str')
        val = np.unique(temp)
        if 'nan' in val:
            true_classes = list(set(val) - set(['nan']))
            num_classes = len(true_classes)
            indices = [i for i, value in enumerate(list(temp)) if value == 'nan']            
            to_replace_by = np.random.randint(low=0, high=num_classes, size=len(indices))
            dataset.iloc[indices, i] = to_replace_by
            
    return dataset

In [4]:
def clean_dataset(dataset_name, data_type):
    print("<---------- Cleaning {} dataset ---------->".format(dataset_name))
    data_config = load_data_config(data_type, dataset_name)
    raw_data = load_dataset(dataset_name, data_type, state='raw')
    
    print("=> Seperating numeric and non_numeric columns ...")
    data = seperate_real_and_categorical_columns(raw_data)

    print("=> Dropping direct identifiers ...")
    direct_identifiers = data_config['identifiers']
    data = data.drop(direct_identifiers, axis=1)
    
    print("=> Replacing NaN values by column mean ...")
    num_real = data_config['num_real']
    data = replace_nan_by_mean(data, num_real=num_real)

    print("=> Shifting the target column to the last index ...")
    target_column = data_config['target_column']
    data = seperate_out_target_variable(data_config, data, target_column)
        
    print("=> Saving cleaned dataset ...")
    save_filepath = join(DATA_DIR, data_type, dataset_name, "clean_data.csv")
    data.to_csv(save_filepath, index=False)
    
    return

### Loading data config

In [5]:
dataset_name = 'lacity'
data_type = 'real'

In [7]:
data_config = load_data_config(data_type, dataset_name)

### Loading raw data

In [8]:
raw_data = load_dataset(dataset_name, data_type, state='raw')

In [9]:
raw_data.head()

,Row ID,Year,Department Title,Payroll Department,Record Number,Job Class Title,Employment Type,Hourly or Event Rate,Projected Annual Salary,Q1 Payments,...,MOU Title,FMS Department,Job Class,Pay Grade,Average Health Cost,Average Dental Cost,Average Basic Life,Average Benefit Cost,Benefits Plan,Job Class Link
0,3-1000027830ctFu,2013,Police (LAPD),4301.0,1000027830,Police Detective II,Full Time,53.16,110998.08,24931.20,...,POLICE OFFICERS UNIT,70,2223,2,11651.40,898.08,191.04,12740.52,Police,http://per.lacity.org/perspecs/2223.pdf
1,3-1000155488ctFu,2013,Police (LAPD),4302.0,1000155488,Clerk Typist,Full Time,23.77,49623.67,11343.96,...,CLERICAL UNIT,70,1358,0,10710.24,405.24,11.40,11126.88,City,http://per.lacity.org/perspecs/1358.pdf
2,3-1000194958ctFu,2013,Police (LAPD),4301.0,1000194958,Police Sergeant I,Full Time,60.80,126950.40,24184.00,...,POLICE OFFICERS UNIT,70,2227,1,11651.40,898.08,191.04,12740.52,Police,http://per.lacity.org/perspecs/2227.pdf
3,3-1000198958ctFu,2013,Zoo,8701.0,1000198958,Light Equipment Operator,Full Time,30.23,63126.77,15319.59,...,EQUIP.OPER. & LABOR,87,3523,0,10710.24,405.24,11.40,11126.88,City,http://per.lacity.org/perspecs/3523.pdf
4,3-1000232317ctFu,2013,Police (LAPD),4301.0,1000232317,Police Lieutenant I,Full Time,60.98,127326.24,29391.20,...,POLICE OFFICERS UNIT,70,2232,1,11651.40,898.08,191.04,12740.52,Police,http://per.lacity.org/perspecs/2232.pdf


### Seperating real and categorical attributes

In [10]:
real_cat_separated_data = seperate_real_and_categorical_columns(raw_data)

In [11]:
real_cat_separated_data.head()

,Year,Payroll Department,Hourly or Event Rate,Projected Annual Salary,Q1 Payments,Q2 Payments,Q3 Payments,Q4 Payments,Payments Over Base Pay,Total Payments,...,Benefits Plan,% Over Base Pay,MOU,MOU Title,Row ID,Job Class Link,Record Number,Employment Type,Department Title,Job Class Title
0,2013,4301.0,53.16,110998.08,24931.20,29181.61,26545.80,29605.30,4499.12,110263.91,...,Police,0.00%,24,POLICE OFFICERS UNIT,3-1000027830ctFu,http://per.lacity.org/perspecs/2223.pdf,1000027830,Full Time,Police (LAPD),Police Detective II
1,2013,4302.0,23.77,49623.67,11343.96,13212.37,11508.36,13442.53,1844.82,49507.22,...,City,0.00%,3,CLERICAL UNIT,3-1000155488ctFu,http://per.lacity.org/perspecs/1358.pdf,1000155488,Full Time,Police (LAPD),Clerk Typist
2,2013,4301.0,60.80,126950.40,24184.00,28327.20,28744.20,33224.88,13192.43,114480.28,...,Police,0.00%,24,POLICE OFFICERS UNIT,3-1000194958ctFu,http://per.lacity.org/perspecs/2227.pdf,1000194958,Full Time,Police (LAPD),Police Sergeant I
3,2013,8701.0,30.23,63126.77,15319.59,17762.14,15601.85,18233.32,5924.08,66916.90,...,City,0.00%,4,EQUIP.OPER. & LABOR,3-1000198958ctFu,http://per.lacity.org/perspecs/3523.pdf,1000198958,Full Time,Zoo,Light Equipment Operator
4,2013,4301.0,60.98,127326.24,29391.20,36591.20,32904.81,37234.03,18034.53,136121.24,...,Police,0.00%,24,POLICE OFFICERS UNIT,3-1000232317ctFu,http://per.lacity.org/perspecs/2232.pdf,1000232317,Full Time,Police (LAPD),Police Lieutenant I


### Removing direct identifiers

In [12]:
direct_identifiers = data_config['identifiers']

In [13]:
data = real_cat_separated_data.drop(direct_identifiers, axis=1)

In [14]:
data.head()

,Year,Payroll Department,Hourly or Event Rate,Projected Annual Salary,Q1 Payments,Q2 Payments,Q3 Payments,Q4 Payments,Payments Over Base Pay,Total Payments,...,Average Basic Life,Average Benefit Cost,Pay Grade,Benefits Plan,% Over Base Pay,MOU,MOU Title,Employment Type,Department Title,Job Class Title
0,2013,4301.0,53.16,110998.08,24931.20,29181.61,26545.80,29605.30,4499.12,110263.91,...,191.04,12740.52,2,Police,0.00%,24,POLICE OFFICERS UNIT,Full Time,Police (LAPD),Police Detective II
1,2013,4302.0,23.77,49623.67,11343.96,13212.37,11508.36,13442.53,1844.82,49507.22,...,11.40,11126.88,0,City,0.00%,3,CLERICAL UNIT,Full Time,Police (LAPD),Clerk Typist
2,2013,4301.0,60.80,126950.40,24184.00,28327.20,28744.20,33224.88,13192.43,114480.28,...,191.04,12740.52,1,Police,0.00%,24,POLICE OFFICERS UNIT,Full Time,Police (LAPD),Police Sergeant I
3,2013,8701.0,30.23,63126.77,15319.59,17762.14,15601.85,18233.32,5924.08,66916.90,...,11.40,11126.88,0,City,0.00%,4,EQUIP.OPER. & LABOR,Full Time,Zoo,Light Equipment Operator
4,2013,4301.0,60.98,127326.24,29391.20,36591.20,32904.81,37234.03,18034.53,136121.24,...,191.04,12740.52,1,Police,0.00%,24,POLICE OFFICERS UNIT,Full Time,Police (LAPD),Police Lieutenant I


In [15]:
all_attributes = data.columns

In [16]:
all_attributes

Index(['Year', 'Payroll Department', 'Hourly or Event Rate',
       'Projected Annual Salary', 'Q1 Payments', 'Q2 Payments', 'Q3 Payments',
       'Q4 Payments', 'Payments Over Base Pay', 'Total Payments', 'Base Pay',
       'Permanent Bonus Pay', 'Longevity Bonus Pay', 'Temporary Bonus Pay',
       'Lump Sum Pay', 'Overtime Pay', 'Other Pay & Adjustments',
       'Other Pay (Payroll Explorer)', 'FMS Department', 'Job Class',
       'Average Health Cost', 'Average Dental Cost', 'Average Basic Life',
       'Average Benefit Cost', 'Pay Grade', 'Benefits Plan', '% Over Base Pay',
       'MOU', 'MOU Title', 'Employment Type', 'Department Title',
       'Job Class Title'],
      dtype='object')

In [21]:
first_categorical_attribute = 'Pay Grade'
num_real = list(all_attributes).index(first_categorical_attribute)
real_attributes = list(all_attributes)[:num_real]

In [22]:
cat_attributes = list(set(all_attributes) - set(real_attributes))
num_cat = len(cat_attributes)

In [23]:
assert num_real + num_cat == len(all_attributes)

In [24]:
num_real, num_cat

(24, 8)

* Please update `num_real` and `num_cat` values in the data config

### Removing NaN values

In [25]:
data = replace_nan_by_mean(data, num_real=data_config['num_real'])

In [26]:
data.head()

,Year,Payroll Department,Hourly or Event Rate,Projected Annual Salary,Q1 Payments,Q2 Payments,Q3 Payments,Q4 Payments,Payments Over Base Pay,Total Payments,...,Average Basic Life,Average Benefit Cost,Pay Grade,Benefits Plan,% Over Base Pay,MOU,MOU Title,Employment Type,Department Title,Job Class Title
0,2013,4301.0,53.16,110998.08,24931.20,29181.61,26545.80,29605.30,4499.12,110263.91,...,191.04,12740.52,2,Police,0.00%,24,POLICE OFFICERS UNIT,Full Time,Police (LAPD),Police Detective II
1,2013,4302.0,23.77,49623.67,11343.96,13212.37,11508.36,13442.53,1844.82,49507.22,...,11.40,11126.88,0,City,0.00%,3,CLERICAL UNIT,Full Time,Police (LAPD),Clerk Typist
2,2013,4301.0,60.80,126950.40,24184.00,28327.20,28744.20,33224.88,13192.43,114480.28,...,191.04,12740.52,1,Police,0.00%,24,POLICE OFFICERS UNIT,Full Time,Police (LAPD),Police Sergeant I
3,2013,8701.0,30.23,63126.77,15319.59,17762.14,15601.85,18233.32,5924.08,66916.90,...,11.40,11126.88,0,City,0.00%,4,EQUIP.OPER. & LABOR,Full Time,Zoo,Light Equipment Operator
4,2013,4301.0,60.98,127326.24,29391.20,36591.20,32904.81,37234.03,18034.53,136121.24,...,191.04,12740.52,1,Police,0.00%,24,POLICE OFFICERS UNIT,Full Time,Police (LAPD),Police Lieutenant I


### Update target and classifier label columns

* [ADULT] `target = "hours-per-week"` and `classifier_label = "income"` in data config
* [LACITY] `target = "Projected Annual Salary"` and `classifier_label = "Employment Type"` in data config

### Additional preprocessing (only for LA-City)

In [27]:
def make_column_with_uniform_datatype(data, column):
    values = np.array(data[column])
    A = [type(x) for x in values]

    data_types = defaultdict(list)
    for i, x in enumerate(A):
        data_types[x.__name__].append(i)

    if len(data_types) > 1:

        dominant_data_type = list(data_types.keys())[0]
        for key in data_types.keys():
            if len(data_types[key]) >= len(data_types[dominant_data_type]):
                dominant_data_type = key
        try:
            values = values.astype(dominant_data_type)
        except:
            # handle the case of column='% Over Base Pay'
            str_values = [np.ceil(float(x[:-1])) for x in values[np.array(data_types['str'])]]
            int_values = list(values[np.array(data_types['int'])])
            values =  np.array(str_values + int_values)
            values = values.astype(dominant_data_type)
    
    data[column] = list(values)
    return data

In [28]:
for attr in tqdm(all_attributes):
    data = make_column_with_uniform_datatype(data, attr)

100%|██████████| 32/32 [00:09<00:00,  3.52it/s]


### Save the cleaned dataset

In [29]:
save_filepath = join(DATA_DIR, data_type, dataset_name, "clean_data.csv")
data.to_csv(save_filepath, index=False)

### Save subset of the dataset (only for LA-City dataset)

In [30]:
data = sample_subset_of_dataset(data, num_examples_reqd=18000)

In [31]:
save_filepath = join(DATA_DIR, data_type, dataset_name, "clean_subset_data.csv")
data.to_csv(save_filepath, index=False)

### Basic data analysis

In [32]:
num_real, num_cat = len(real_attributes), len(cat_attributes)

In [33]:
visualise = False

In [34]:
for attr in cat_attributes:
    try:
        num_bins = len(np.unique(np.array(data[attr])))
        print("=> Number of unique bins for {} is {}".format(attr, num_bins))
    except:
        print(attr)
    
    if visualise:
        plt.figure(figsize=(5 * num_bins, 3 * num_bins))
        sns.countplot(data[attr])
        plt.grid()
        plt.title("Distribution for {}".format(attr), fontsize= 2 * num_bins);

=> Number of unique bins for Benefits Plan is 8
=> Number of unique bins for Pay Grade is 48
=> Number of unique bins for % Over Base Pay is 45
=> Number of unique bins for MOU is 90
=> Number of unique bins for MOU Title is 360
=> Number of unique bins for Employment Type is 3
=> Number of unique bins for Department Title is 85
=> Number of unique bins for Job Class Title is 1147
